<a href="https://colab.research.google.com/github/xainasx/News-Clusterize/blob/main/News_Clusterizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install feedparser
!pip install nltk
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 KB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6065 sha256=2de3f3ac4cf5e8884568ca2828c94b9a9517e6627a5a962edb572cfd84889f84
  Stored in directory: /root/.cache/pip/wheels/65/7a/a7/78c287f64e401255dff4c13fdbc672fed5efbfd21c530114e1
Successfully built sgmllib3k
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import feedparser
import pandas as pd
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.stem import WordNetLemmatizer
from collections import Counter

In [ ]:
sites = ["https://feeds.bbci.co.uk/sport/rss.xml",
            "https://www.espn.com/espn/rss/news",
            "http://rss.cnn.com/rss/edition_sport.rss",
            "https://www.wired.com/feed/rss",
            "http://feeds.bbci.co.uk/news/technology/rss.xml",
            "https://gizmodo.com/rss",
            "https://inhabitat.com/environment/feed/",
            "http://feeds.bbci.co.uk/news/science_and_environment/rss.xml",
            "https://feeds.feedburner.com/climatecentral/djoo",
            "https://www.climaterealityproject.org/rss.xml"
           ]

In [ ]:
title = []
for feed in sites:
    feed_data = feedparser.parse(feed)
    for entry in feed_data.entries:
        title.append(entry.title)
print(len(title)) # to check whether is 100 above

368


In [ ]:
# Clean the titles
def cleaning(title):
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    filtered_str = []
    for doc in title:
        tokens = word_tokenize(doc)
        tmp = " ".join(lemmatizer.lemmatize(w) for w in tokens if w not in stop_words)
        filtered_str.append(tmp)
    filtered_str = [i.replace("'", "") for i in filtered_str]
    return filtered_str

pre_proc = cleaning(title)
pre_proc

['Pistorius denied parole murder girlfriend Steenkamp',
 'Racism claim Vaughan found not proved ',
 'Premier League build-up news conference',
 'Wimbledon lift ban Russian & Belarusian player',
 'Bulked-up Joshua career heaviest Franklin',
 'Renfroe make ridiculous  no-look catch debut',
 'Muir Reekie part long-serving coach Young',
 'A ‘ monumental week ’ exactly Liverpool need',
 'Listen : IPL - Gujarat Titans v Chennai Super Kings - Stokes & Moeen play',
 'The hearing closure - Rafiq',
 'Rafiq feel vindicated  hearing',
 'Guardiola may risk Haaland fitness Liverpool',
 'Alonso top rain-affected Australian GP second practice',
 'Who watch new England era begin',
 'Why De Zerbi set one best ',
 'Sutton s Premier League prediction v DMA S frontman & Everton fan Tommy ODell',
 'England Netball launch woman s health initiative',
 'Ireland secure first victory Bangladesh tour',
 'Women s Tour 2023 cancelled amid funding issue',
 'Five thing look Champions Cup',
 'Potter praise Mount attit

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(pre_proc)

model = KMeans(3, max_iter=500, random_state=42)
model.fit(X)

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(max_iter=500, n_clusters=3, random_state=42)

In [ ]:
labels = model.labels_
cluster_counts = Counter(labels)

# Print the number of documents in each cluster
for cluster_label, count in cluster_counts.items():
    print(f"Cluster {cluster_label} has {count} titles")

Cluster 0 has 306 titles
Cluster 1 has 34 titles
Cluster 2 has 28 titles


In [ ]:
  while True:
    ask = input("\nDo you want to search? 'y' or 'n': ")
    if ask == 'y':
        inp = input("Write here: ")
        inp = cleaning([inp])
        Y = vectorizer.transform(inp)
        prediction = model.predict(Y.toarray())
        categories = {0: 'Sports', 1: 'Climate', 2: 'Technology'}
        predict_category = categories[prediction[0]]
        print("Your search string prediction: ", predict_category)
    elif ask == 'n':
        break
    else:
        print("Invalid input. Please enter 'y' or 'n'.")

Your search string prediction:  Sports
